# Modeling Crop Yield Anomaly
## Python modules

In [1]:
## import warnings
import time
import math
import os
import glob
from pyhere import here
from datetime import date

import numpy as np
import pandas as pd
import geopandas

import pyarrow
import itertools
import multiprocessing
import p_tqdm

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split, KFold, LeaveOneGroupOut, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import r2_score
from scipy.stats import spearmanr,  pearsonr

In [2]:
def str2bool(string):
    return string.lower() in ("yes", "true", "t", "1")

In [3]:
data_dir = here("data")
directory = here("data", "random_features", "summary")
today = date.today().strftime("%Y-%m-%d")
files = os.listdir(directory)
files = [f for f in files if f not in ('.gitkeep', '.ipynb_checkpoints')]
len(files)

56

In [4]:
paramlist = list(itertools.product(files, [True, False]))
len(paramlist)

112

In [5]:
def model(params):
    #########################################     SET PARAMS    #########################################
    file         = params[0]
    hot_encode   = params[1]
    f            = file.split(sep="_")
    satellite    = f[0]
    bands        = f[1].replace("bands-", "")
    country_code = f[2]
    points       = f[3].replace("k-points", "")
    num_features = f[4].replace("-features", "")
    yrs          = f[5].replace("yr-", "").split(sep="-")
    mns          = f[6].replace("mn-", "").split(sep="-")
    limit_months = str2bool(f[7].replace("lm-", ""))
    crop_mask    = str2bool(f[8].replace("cm-", ""))
    weighted_avg = str2bool(f[9].replace("wa-", ""))
    years        = range(int(yrs[0]), int(yrs[1])+1)
    month_range  = list(range(int(mns[0]), int(mns[1])+1))
    
    #########################################     READ DATA    #########################################
    fn = f"{directory}/{file}"
    features = pd.read_feather(fn)
     
    drop_cols = ['district', 'year', 'yield_mt', "crop_perc"]
            
    if weighted_avg:
        drop_cols.remove("crop_perc")
    else:
        pass

    #########################################     CALCULATE ANOMALY   #########################################
    features['yield_mt'] = np.log10(features['yield_mt'] + 1)
    features.set_index(['year', 'district'], inplace=True)
    var_cols = features.columns
    features = features[var_cols] - features.groupby(['district'], as_index=True)[var_cols].transform('mean')
    features.reset_index(drop=False, inplace=True)

    crop_yield = features.copy().loc[:, tuple(drop_cols)]
    
    if hot_encode:
        drop_cols.remove("district")
        features = pd.get_dummies(features, columns=["district"], drop_first=False)
    else:
        pass
    
    #########################################     SPLIT DATA    #########################################
    x_all = features.drop(drop_cols, axis = 1) 
    y_all = features.yield_mt
    g_all = features.year.ravel()
    
    x_train, x_test,\
    y_train, y_test,\
    g_train, g_test = train_test_split(x_all, y_all, g_all, test_size = 0.2, random_state = 0)

    #########################################     K-FOLD CV    ###########################################
    kfold = KFold()
    ridge = Ridge()
    parameters = {'alpha': np.logspace(-8, 8, base = 10, num = 17)}
    ### GRID SEARCH - FINDING BEST REGULARIZATION PARAMETER
    ridge_kfold_reg = GridSearchCV(ridge, parameters, scoring = 'r2', cv = kfold)
    ridge_kfold_reg.fit(x_train, y_train)
    best_kfold_model = ridge_kfold_reg.best_estimator_
    ### CV PREDICT - PREDICTING WITH BEST HYPERPARAMETER
    kfold_val_predictions = cross_val_predict(best_kfold_model, X = x_train, y = y_train, cv = kfold)   
    ### TRAIN BEST MODEL AND PREDICT
    best_kfold_model.fit(x_train, y_train)
    y_pred_train = np.maximum(best_kfold_model.predict(x_train), 0)
    y_pred_test  = np.maximum(best_kfold_model.predict(x_test), 0)

    #########################################     LOGO CV    ###########################################
    logo = LeaveOneGroupOut()
    ridge = Ridge()
    parameters = {'alpha': np.logspace(-8, 8, base = 10, num = 17)}
    ### GRID SEARCH - FINDING BEST REGULARIZATION PARAMETER
    ridge_logo_reg = GridSearchCV(ridge, parameters, scoring = 'r2', cv = logo)
    ridge_logo_reg.fit(x_all, y_all, groups = g_all)
    best_logo_model = ridge_logo_reg.best_estimator_
    ### CV PREDICT - PREDICTING WITH BEST HYPERPARAMETER
    logo_val_predictions = cross_val_predict(best_logo_model, X = x_all, y = y_all, groups = g_all,  cv = logo)   
    
    #########################################     SAVE RESULTS    #########################################
    d = {
        'country': country_code,
        'satellite': satellite,
        'bands': bands,
        'num_features': num_features,
        'points': points, 
        'month_range': f'{min(month_range)}-{max(month_range)}',
        
        'limit_months': limit_months,
        'crop_mask': crop_mask,
        'weighted_avg': weighted_avg,
        'hot_encode': hot_encode,
        
        'total_n': len(x_all),
        'train_n': len(x_train),
        'test_n': len(x_test),
        
        'kfold_best_reg_param': list(ridge_kfold_reg.best_params_.values())[0],
        'kfold_mean_of_val_R2s': ridge_kfold_reg.best_score_,
        'kfold_val_R2': r2_score(y_train, kfold_val_predictions),
        'kfold_val_r' : pearsonr(kfold_val_predictions, y_train)[0],
        'kfold_val_r2' : pearsonr(kfold_val_predictions, y_train)[0] ** 2,
        
        'kfold_train_R2': r2_score(y_train, y_pred_train),
        'kfold_train_r': pearsonr(y_pred_train, y_train)[0],
        'kfold_train_r2': pearsonr(y_pred_train, y_train)[0] ** 2,
        
        'kfold_test_R2': r2_score(y_test, y_pred_test),
        'kfold_test_r': pearsonr(y_pred_test, y_test)[0],
        'kfold_test_r2': pearsonr(y_pred_test, y_test)[0] ** 2,
                
        'logo_best_reg_param': list(ridge_logo_reg.best_params_.values())[0],      
        'logo_mean_of_val_R2s' : ridge_logo_reg.best_score_,
        'logo_val_R2' : r2_score(y_all, logo_val_predictions),
        'logo_val_r' : pearsonr(logo_val_predictions, y_all)[0],
        'logo_val_r2' : pearsonr(logo_val_predictions, y_all)[0] ** 2
    }
    return pd.DataFrame(data=d, index=[0])

In [6]:
%%time     
##### With progress bar
workers = os.cpu_count()
if __name__ == "__main__":
    output = []
    for result in p_tqdm.p_map(model, paramlist, num_cpus=workers):
        output.append(result)
    results = pd.concat(output).reset_index(drop=True)
    today = date.today().strftime("%Y-%m-%d")
    file_name = f'anomaly_results_{today}.csv'
    print(f"Saving results as: {file_name}\n\n")           
    results.to_csv(here("data","results", file_name))

  0%|          | 0/112 [00:00<?, ?it/s]

Saving results as: anomaly_results_2022-11-02.csv


CPU times: user 1.16 s, sys: 618 ms, total: 1.78 s
Wall time: 47min 12s


In [7]:
results.iloc[:, 14:]

,kfold_mean_of_val_R2s,kfold_val_R2,kfold_val_r,kfold_val_r2,kfold_train_R2,kfold_train_r,kfold_train_r2,kfold_test_R2,kfold_test_r,kfold_test_r2,logo_best_reg_param,logo_mean_of_val_R2s,logo_val_R2,logo_val_r,logo_val_r2
0,0.160962,0.180453,0.425619,0.181152,0.095062,0.359572,0.129292,0.031125,0.278669,0.077656,1.000000e+05,-0.424331,-0.087300,-0.371845,0.138268
1,0.217341,0.228561,0.491287,0.241363,0.148435,0.458271,0.210012,0.013648,0.355144,0.126127,1.000000e-05,0.206067,0.490617,0.716921,0.513976
2,0.259984,0.257457,0.511569,0.261703,0.133033,0.449546,0.202091,-0.023558,0.210403,0.044269,1.000000e+08,-0.435523,-0.107721,-0.545970,0.298084
3,0.274492,0.269965,0.550311,0.302843,0.219811,0.580455,0.336928,0.017604,0.319307,0.101957,1.000000e-02,0.344305,0.628017,0.797162,0.635467
4,0.162360,0.181976,0.427333,0.182614,0.095714,0.360751,0.130141,0.025506,0.277376,0.076938,1.000000e+05,-0.424088,-0.087104,-0.369924,0.136844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,0.359263,0.379954,0.617376,0.381153,0.159652,0.472218,0.222990,0.088620,0.439192,0.192890,1.000000e-05,0.628034,0.790992,0.890984,0.793853
108,0.178799,0.197700,0.456773,0.208641,0.230141,0.596473,0.355780,0.043578,0.222264,0.049401,1.000000e+08,-0.545810,-0.076961,-0.665777,0.443259
109,0.244813,0.268051,0.535299,0.286545,0.317920,0.669120,0.447722,0.076044,0.307685,0.094670,1.000000e-08,0.121553,0.410247,0.676122,0.457141
110,0.106557,0.126796,0.356104,0.126810,0.054460,0.273444,0.074771,0.039146,0.273117,0.074593,1.000000e+08,-0.431657,-0.107730,-0.545763,0.297857


In [ ]:
# %%time    
##### No progress bar
# if __name__ == "__main__":
#     with multiprocessing.Pool(processes=os.cpu_count()) as pool:
#         output = []
#         for result in pool.imap_unordered(model, paramlist, chunksize=2):
#             output.append(result)
#     results = pd.concat(output).reset_index(drop=True)
#     file_name = f'results_{today}.csv'
#     print(f"Saving results as: {file_name}\n\n")           
#     results.to_csv(here("data","results", file_name))